# Install and init DVC

Prerequisites: 
-  DVC and requirements.txt packages installed (if not - check README.md file for instructions)
-  A project repository is a Git repo 

## Install with pip

In [ ]:
# !pip install dvc==2.10.1

## Checkout branch `experiments`

In [ ]:
!git checkout -b experiments

## Initialize DVC

References: 
- https://dvc.org/doc/get-started/initialize 

In [ ]:
!dvc init

## Commit changes

In [ ]:
%%bash

git add .
git commit -m "Initialize DVC"

## Review Files and Directories created by DVC

In [ ]:
!ls -a .dvc 

In [ ]:
!cat .dvc/.gitignore

# Quick Tour of DVC features

## Data Versioning

In [1]:
# Get data 

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
list(data.target_names)
data.frame.to_csv('data/iris.csv', index=False)

In [2]:
# Look on data

data.frame.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
%%bash

du -sh data/*

## Add file under DVC control

In [ ]:
%%bash

dvc add data/iris.csv

In [ ]:
!du -sh data/*

In [ ]:
!git status -s data/

In [ ]:
%%bash

git add .
git commit -m "Add a source dataset"

### What is DVC-file?

Data file internals


>    If you take a look at the DVC-file, you will see that only outputs are defined in outs. 
    In this file, only one output is defined. The output contains the data file path in the repository and md5 cache.
    This md5 cache determines a location of the actual content file in DVC cache directory .dvc/cache
    >> Output from DVC-files defines the relationship between the data file path in a repository and the path in a cache directory. See also DVC File Format



(c) dvc.org https://dvc.org/doc/tutorial/define-ml-pipeline

In [ ]:
!cat data/iris.csv.dvc

## Create and Reproduce ML pipelines 

Stages 
- extract features 
- split dataset 
- train 
- evaluate 


### Add a pipeline stage with 'dvc stage add'

In [ ]:
!dvc stage add \
    -n feature_extraction \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    python src/featurization.py

In [ ]:
!cat dvc.yaml

### Add split train/test stage

In [ ]:
!dvc stage add \
    -n split_dataset \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    python src/split_dataset.py --test_size 0.4

In [ ]:
!cat dvc.yaml

### Add train stage

In [ ]:
!dvc stage add \
    -n train \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    python src/train.py

In [ ]:
!cat dvc.yaml

### Add evaluate stage

In [ ]:
!dvc stage add \
    -n evaluate \
    -d src/train.py \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -m data/eval.txt \
    python src/evaluate.py

In [ ]:
!cat dvc.yaml

### Run DVC pipeline (all stages)

In [ ]:
!dvc repro

In [ ]:
!ls 

In [ ]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

In [ ]:
!git status -s

In [ ]:
%%bash
git add .
git commit -m "Create DVC pipeline"

### Reproduce pipeline

In [ ]:
!dvc repro split_dataset

## Collaborate on ML Experiments 

### Specify remote storage (local ~ /tmp/dvc)


In [ ]:
!dvc remote add -d local /tmp/dvc

### Push features to remote storage

In [ ]:
!dvc push

### Create tag `experiment-1`

In [ ]:
!git tag -a experiment-1 -m "experiment-1"

### Checkout into your teammate experiment state

In [ ]:
%%bash 

git checkout experiment-1
dvc checkout

### Check Metrics

In [ ]:
!dvc metrics show

### Reproduce experiment

In [ ]:
# Nothing to reproduce
!dvc repro

In [ ]:
!dvc repro -f

In [ ]:
# Check Metrics
!dvc metrics show